# Data cleaning for tabular clinical data

**Key functions we will use:**
- `pd.read_csv()` → load data from CSV files into a DataFrame.
- `.head()` → view the first rows of a DataFrame.
- `.info()` → summary of the DataFrame (columns, types, missing values).
- `.describe()` → summary statistics for numeric/categorical columns.
- `.isna()` / `.isnull()` → identify missing values.
- `.replace()` → replace values in the DataFrame.
- `.drop_duplicates()` → remove duplicate rows.
- `.dropna()` → remove missing values.
- `.astype()` → convert column types.
- `.merge()` → join two DataFrames.
- `.plot()` / `matplotlib` / `seaborn` → visualize data.

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
patients = pd.read_csv('data/patients.csv')
labs = pd.read_csv('data/lab_data.csv')

In [ ]:
# show first few rows
patients.head()

In [ ]:
# show labs data
labs.head()

## Joining datasets

- Use `pd.merge()` to combine datasets on a key (`patient_id`)
- `how='left'` keeps all patient rows and adds lab values if available
- (Use `how='inner'` to keep only rows present in both tables or `how='outer'` to keep all rows from both tables)

In [ ]:
# merge patients with labs
patients = pd.merge(patients, labs, on='patient_id', how='left')

In [ ]:
# show first few rows after merge
patients.head()

## Quick exploration: `.info()` and `.describe()`

- `df.info()` shows column names, data types, and non-null counts. It's the fastest way to spot missing values and wrong dtypes
- `df.describe(include='all')` gives summary statistics for numeric and non-numeric columns

In [ ]:
patients.info()

In [ ]:
patients.describe(include='all').transpose()

## Converting columns to correct data types

- Numeric columns stored as strings prevent calculations. We use `pd.to_numeric(..., errors='coerce')` to convert values to numbers; invalid parses become NaN (so we can handle them deliberately).
- Date columns are converted with `pd.to_datetime(..., errors='coerce')` so we can do date arithmetic.
- Categorical variables can be cast to `'category'` dtype to save memory and make operations explicit.

We'll convert `age`, `height` to numeric, and attempt to parse admission/discharge dates.

In [ ]:
# Convert 'age' to int64
patients['age'] = pd.to_numeric(patients['age'], errors='coerce').astype('Int64')

# Convert dates to datetime
patients['admission_date'] = pd.to_datetime(patients['admission_date'], errors='coerce')
patients['discharge_date'] = pd.to_datetime(patients['discharge_date'], errors='coerce')

In [ ]:
# Show dtypes after conversion
patients.dtypes

## Visualizing distributions

- Visualizing numeric columns helps spot outliers and understand distributions.
- We can use barplots and boxplots.
- We focus on `weight`, `blood_pressure`, and `age` here.

In [ ]:
# Boxplots for numeric columns to spot outliers for weight, blood_pressure in separate plots
plt.figure(figsize=(12,7))
plt.subplot(1,3,1)
sns.boxplot(y=patients['weight'])
plt.title('Weight Distribution')
plt.subplot(1,3,2)
sns.boxplot(y=patients['blood_pressure'])
plt.title('Blood Pressure Distribution')
plt.subplot(1,3,3)
sns.boxplot(y=patients['age'])
plt.title('Age Distribution')
plt.tight_layout()
plt.show()

## Handling outliers

- Outliers should be inspected: are they data entry mistakes or true extreme clinical cases?
- For obvious errors (e.g., weight = 500 kg or height = 300 cm), we may drop or cap them. Here we show a conservative filters example to remove implausible values.
- The more data-driven IQR method adapts to the actual distribution. We will apply both methods.

In [ ]:
# Count rows before filtering
before = len(patients)

### Example conservative filtering: remove implausible physiological values

- Height: keep between 100 cm and 250 cm
- Blood pressure: keep between 40 mmHg and 250 mmHg
- We allow NaNs to remain for now (will handle missingness later)

In [ ]:
patients = patients[(patients['height'].isna()) | ((patients['height'] > 100) & (patients['height'] < 250))]
patients = patients[(patients['blood_pressure'].isna()) | ((patients['blood_pressure'] > 40) & (patients['blood_pressure'] < 250))]

### Example IQR-based filtering for weight

- Calculate Q1, Q3, and IQR for weight
- Define lower and upper bounds as Q1 - 1.5*IQR and Q3 + 1.5*IQR
- Filter rows outside these bounds
- This method is more data-driven and adapts to the actual distribution
- We allow NaNs to remain for now (will handle missingness later)

In [ ]:
Q1 = patients['weight'].quantile(0.25)
Q3 = patients['weight'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
patients = patients[((patients['weight'] >= lower_bound) & (patients['weight'] <= upper_bound))]

In [ ]:
# Count rows after filtering and print both counts
after = len(patients)
print(f"Rows before: {before}, after outlier filter: {after}")

## Detect missing values

Real-world datasets contain often missing values. We count missing values per column to understand the extent of missingness. Afterwards we will handle them appropriately (depending on the analysis, we may drop rows, impute values, or flag missingness).

In [ ]:
# Show missing counts per column
patients.isna().sum()

### Method 1: Drop rows with missing values in critical columns (e.g., admission_date, discharge_date)

In [ ]:
# Drop rows missing admission or discharge dates
patients = patients.dropna(subset=['admission_date', 'discharge_date', 'height'])

### Method 2: Impute missing data (e.g., blood_pressure) with mean

In [ ]:
# Impute missing blood_pressure with mean
mean_bp = patients['blood_pressure'].mean()
patients['blood_pressure'] = patients['blood_pressure'].fillna(mean_bp)

# Impute albumin_g_dl with mean
mean_albumin = patients['albumin_g_dl'].mean()
patients['albumin_g_dl'] = patients['albumin_g_dl'].fillna(mean_albumin)

In [ ]:
# Show missing counts per column
patients.isna().sum()

## Detecting and removing duplicates

- Duplicates (identical rows) can inflate counts and bias results. Use `df.duplicated()` to detect and `df.drop_duplicates()` to remove.
- If duplicates are partial (same patient with different timestamps), domain knowledge is required to deduplicate safely.

In [ ]:
# Count duplicate rows (exact duplicates)
dup_count = patients.duplicated().sum()
print("Exact duplicate rows:", dup_count)

# Example: show duplicates if any (first few)
if dup_count > 0:
    display(patients[patients.duplicated(keep=False)].head())

# Remove exact duplicates
patients = patients.drop_duplicates()

### Save cleaned dataset

In [ ]:
# Save cleaned data to new CSV
patients.to_csv('data/patients_cleaned.csv', index=False)

## 8 Exercise — Data Cleaning

Please complete the following tasks:

1. Handle outliers in `age` by setting rows where age is less than 18 and more than 100 to NaN.
2. Count how many patients have missing values in `age`.
3. Impute missing `age` values with the median age. (use `.median()`)
4. Plot the `age` distribution after cleaning and briefly interpret whether values look plausible.